<a href="https://colab.research.google.com/github/Awoyawson/Artificial-Neutral-Network/blob/main/Cross_validation_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''To implement K-fold cross-validation, we use a scikit_learn wrapper in Keras: KerasClassifier. 
In other words, we use Keras to build the model and use scikit_learn for cross-validation. '''

'To implement K-fold cross-validation, we use a scikit_learn wrapper in Keras: KerasClassifier. \nIn other words, we use Keras to build the model and use scikit_learn for cross-validation. '

In [1]:
import tensorflow as tf
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [3]:
def build_classifier(optimizer):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
  model.add(tf.keras.layers.Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
  model.add(tf.keras.layers.Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

my_model = KerasClassifier(build_fn = build_classifier)

In [5]:
# create dataset
import pandas as pd
dataset = pd.read_csv('Churn_Modelling.csv')  
X = dataset.iloc[:, 3: 13].values
y = dataset.iloc[:, 13].values

from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder() #instantiate an object of the class LabelEncoder
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) #ordinal encoding for column 1

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) #ordinal encoding for column 2

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
ct = ColumnTransformer( #'encoder' is the name of the column transformer
    [('encoder', OneHotEncoder(), [1])],    # The column numbers to be transformed (here is [1] but can be [0, 1, 3])
    remainder='passthrough'                         # Leave the rest of the columns untouched
)
X = np.array(ct.fit_transform(X), dtype=np.float)

X = X[:, 1:]

#Standardise the data (x_standardised = (x - x_mean)/std_dev)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
'''X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) #note that we use the scale set from the training set to transform the test set'''
X = sc.fit_transform(X)

In [6]:
parameters = {'batch_size': [20, 30], 'nb_epoch': [50, 100], 'optimizer': ['adam', 'sgd']}


In [7]:
grid_search = GridSearchCV(estimator=my_model, param_grid=parameters, scoring = 'accuracy', cv = 10)

In [8]:
grid_search = grid_search.fit(X, y)

450/450 [==============================] - 1s 1ms/step - loss: 0.5960 - accuracy: 0.7861


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


500/500 [==============================] - 1s 1ms/step - loss: 0.5743 - accuracy: 0.7931


In [9]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

print(best_parameters)
print(best_accuracy)

{'batch_size': 20, 'nb_epoch': 50, 'optimizer': 'adam'}
0.7962999999999999
